### 평년평균.csv에 평균.csv의 '평균가격(원)' 칼럼으로 대체 

In [2]:
import pandas as pd

# 평년평균.csv 파일 읽기
df_pyeongnyeon = pd.read_csv('평년평균.csv', encoding='utf-8')

# 평균.csv 파일 읽기
df_pyeonggyun = pd.read_csv('평균.csv', encoding='utf-8')

# 두 데이터프레임의 인덱스가 같은지 확인
if df_pyeongnyeon.shape[0] != df_pyeonggyun.shape[0]:
    print("Warning: 두 파일의 행 수가 다릅니다. 결과를 주의해서 확인하세요.")

# '평균가격(원)' 열 업데이트
df_pyeongnyeon['평균가격(원)'] = df_pyeonggyun['평균가격(원)']

df_pyeongnyeon.to_csv('data_preprocessing\train_preprocess/y_v1.csv', index=False)

print('완료')

파일이 성공적으로 업데이트되었습니다. '평년평균_업데이트.csv'를 확인하세요.


### 추가 전처리

In [ ]:
df= pd.read_csv(r'data_preprocessing\train_preprocess/y_v1.csv', encoding='utf-8')

# 조건을 설정 ('품목명', '품종명', '거래단위', '등급'에 대해)
condition = (
    (df['품목명'] == '깐마늘(국산)') &
    (df['품종명'] == '깐마늘(국산)') &
    (df['거래단위'] == '20 kg') &
    (df['등급'] == '상품')
)

# 조건을 만족하는 행에 대해 '평년 평균가격(원)' 값을 '평균가격(원)' 컬럼에 대체
df.loc[condition, '평년 평균가격(원)'] = df.loc[condition, '평균가격(원)']

# 조건을 설정 ('품목명', '품종명', '거래단위', '등급'에 대해)
condition2 = (
    (df['품목명'] == '무') &
    (df['품종명'] == '무') &
    (df['거래단위'] == '20키로상자') &
    (df['등급'] == '상')
)

# 조건을 만족하는 행에 대해 '평년 평균가격(원)' 값을 '평균가격(원)' 컬럼에 대체
df.loc[condition2, '평년 평균가격(원)'] = df.loc[condition2, '평균가격(원)']


df.to_csv('data_preprocessing\train_preprocess/y_v2.csv', index=False)

### LOESS 스무딩 처리

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib.pyplot as plt
import warnings
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

# 마이너스 폰트 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 경고 메시지 무시
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 데이터 로드
df = pd.read_csv(r'data_preprocessing\train_preprocess/y_v2.csv', encoding='utf-8')

# 원본 데이터 복사
df_original = df.copy()

# '시점' 열 처리
def process_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    if isinstance(date_str, str):
        date_str = date_str.replace('상순', '05').replace('중순', '15').replace('하순', '25')
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    return pd.to_datetime(date_str, errors='coerce')

df['시점'] = df['시점'].apply(process_date)
df_original['시점'] = df_original['시점'].apply(process_date)

# LOESS 스무딩 함수 정의
def apply_loess_safe(group, column, frac=0.3):
    if group[column].isna().all() or (group[column] == 0).all():
        return group[column]
    try:
        valid_data = group[~group[column].isin([np.nan, 0])]
        if len(valid_data) < 2:
            return group[column]
        
        smoothed = lowess(
            endog=valid_data[column],
            exog=valid_data['시점'].astype(int),
            frac=frac,
            it=3,
            delta=0.0,
            is_sorted=True,
            return_sorted=False
        )
        
        result = pd.Series(index=group.index, dtype=float)
        result.loc[valid_data.index] = smoothed
        return result
    except Exception as e:
        print(f"Error smoothing group for {column}: {e}")
        return group[column]

# 데이터프레임에 연도 열 추가
df['year'] = df['시점'].dt.year

# 두 열에 대해 LOESS 스무딩 적용
columns_to_smooth = ['평균가격(원)', '평년 평균가격(원)']

for column in columns_to_smooth:
    for (품목, 품종, 등급, 거래단위), group in df.groupby(['품목명', '품종명', '등급', '거래단위']):
        smoothed_values = apply_loess_safe(group, column)
        
        mask = (df['품목명'] == 품목) & (df['품종명'] == 품종) & (df['등급'] == 등급) & (df['거래단위'] == 거래단위)
        df.loc[mask & ((df[column] == 0) | df[column].isna()), column] = smoothed_values[
            (df.loc[mask, column] == 0) | df.loc[mask, column].isna()
        ]

# 결과 확인
for column in columns_to_smooth:
    print(f"\n{column} 스무딩 후 결측치 및 0 값 개수:")
    print((df[column] == 0).sum(), "개의 0 값")
    print(df[column].isnull().sum(), "개의 NaN 값")

# 스무딩 결과 시각화
def plot_smoothed_price_trends(df_original, df_smoothed):
    for (품목, 품종, 등급, 거래단위), group in df_smoothed.groupby(['품목명', '품종명', '등급', '거래단위']):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12))
        
        original_group = df_original[
            (df_original['품목명'] == 품목) & 
            (df_original['품종명'] == 품종) & 
            (df_original['등급'] == 등급) & 
            (df_original['거래단위'] == 거래단위)
        ]
        
        for ax, column in zip([ax1, ax2], columns_to_smooth):
            ax.scatter(original_group['시점'], original_group[column], 
                        alpha=0.5, label='원본 데이터', color='blue', marker='o')
            ax.plot(group['시점'], group[column], 
                     label='스무딩 후', color='red', linestyle='-', linewidth=2)
            
            ax.set_title(f'[{품목} - {품종}] {column} ({등급}, {거래단위}) (2018-2021)')
            ax.set_xlabel('시점')
            ax.set_ylabel(column)
            ax.legend()
            ax.grid(True)
            ax.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

# 스무딩 결과 시각화 함수 호출
plot_smoothed_price_trends(df_original, df)

In [ ]:
df.to_csv('data_preprocessing/train_preprocess/y_v3.csv', index=False)